In [1]:
import os
os.chdir("..")

In [2]:
import pandas as pd


In [3]:
df = pd.read_csv("data/tech_salaries_filtered_no_others.csv")

In [4]:
df

,dedicacion,contrato,salario,anos_de_experiencia,antiguedad_en_la_empresa_actual,anos_en_el_puesto_actual,cuantas_personas_tenes_a_cargo,cantidad_de_personas_en_tu_organizacion,modalidad_de_trabajo,edad,seniority,marvin_rol
0,Full-Time,Staff (planta permanente),3952805.00,3,3,1,2,De 201 a 500 personas,100% remoto,29,Semi-Senior,5
1,Part-Time,Staff (planta permanente),1606000.00,5,2,2,0,De 1001 a 2000 personas,100% remoto,25,Semi-Senior,4
2,Full-Time,Staff (planta permanente),4000000.00,25,3,3,5,De 2 a 10 personas,100% remoto,50,Senior,3
3,Full-Time,Freelance,3000000.00,18,8,6,0,De 2 a 10 personas,100% remoto,41,Senior,3
4,Full-Time,Staff (planta permanente),3953142.06,15,5,4,3,De 201 a 500 personas,100% remoto,47,Senior,2
...,...,...,...,...,...,...,...,...,...,...,...,...
3330,Full-Time,Staff (planta permanente),2100000.00,4,2,2,0,De 2001a 5000 personas,Híbrido (presencial y remoto),36,Semi-Senior,2
3331,Full-Time,Tercerizado (trabajo a través de consultora o ...,1600000.00,1,1,1,0,De 1001 a 2000 personas,Híbrido (presencial y remoto),23,Junior,4
3332,Full-Time,Contractor,3000000.00,3,3,3,3,De 101 a 200 personas,100% remoto,28,Semi-Senior,3
3333,Full-Time,Staff (planta permanente),5000000.00,20,15,4,1,De 201 a 500 personas,Híbrido (presencial y remoto),58,Senior,3


In [6]:
df.cantidad_de_personas_en_tu_organizacion.value_counts()

cantidad_de_personas_en_tu_organizacion
De 11  a 50  personas       594
De 51 a 100 personas        461
De 201 a 500 personas       422
De 101 a 200 personas       412
Más de 10000 personas       368
De 501 a 1000 personas      270
De 2001a 5000 personas      237
De 1001 a 2000 personas     222
De 2 a 10 personas          187
De 5001 a 10000 personas    140
1 (solamente yo)             22
Name: count, dtype: int64

In [7]:
# Definir diccionarios de mapeo
mapeo_cantidad_personas = {
    '1 (solamente yo)': 0,
    'De 2 a 10 personas': 1,
    'De 11  a 50  personas': 2,
    'De 51 a 100 personas': 3,
    'De 101 a 200 personas': 4,
    'De 201 a 500 personas': 5,
    'De 501 a 1000 personas': 6,
    'De 1001 a 2000 personas': 7,
    'De 2001a 5000 personas': 8,
    'De 5001 a 10000 personas': 9,
    'Más de 10000 personas': 10
}

mapeo_modalidad = {
    '100% presencial': 0,
    'Híbrido (presencial y remoto)': 1,
    '100% remoto': 2
}

mapeo_seniority = {
    'Junior': 0,
    'Semi-Senior': 1,
    'Senior': 2,
    'Manager or Above': 3  # Tratar como nivel superior
}

# Aplicar mapeos
df['cantidad_personas_encoded'] = df['cantidad_de_personas_en_tu_organizacion'].map(mapeo_cantidad_personas)
df['modalidad_encoded'] = df['modalidad_de_trabajo'].map(mapeo_modalidad)
df['seniority_encoded'] = df['seniority'].map(mapeo_seniority)

In [8]:
df.drop(columns=['cantidad_de_personas_en_tu_organizacion', 'modalidad_de_trabajo', 'seniority'], inplace=True)

In [9]:
df['contrato'] = df['contrato'].replace({
    'Tercerizado (trabajo a través de consultora o agencia)': 'Contractor',
    'Freelance': 'Contractor',
    "Participación societaria en una cooperativa": 'Contractor',
})

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# 1. Definir columnas a transformar
columnas_categoricas = ['contrato', 'dedicacion']

# 2. Configurar el transformador (sin drop para binarias)
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(
            sparse_output=False,
            handle_unknown='ignore',
            drop=None  # Mantener ambas categorías
        ), columnas_categoricas)
    ],
    remainder='passthrough'
)

# 3. Aplicar transformación
df_encoded = preprocessor.fit_transform(df)

# 4. Obtener nombres de columnas
nombres_columnas = preprocessor.get_feature_names_out(
    input_features=df.columns
)

# 5. Convertir a DataFrame
df_encoded = pd.DataFrame(df_encoded, columns=nombres_columnas)

In [11]:
df_encoded.drop(columns=['onehot__contrato_Contractor', 'onehot__dedicacion_Part-Time'], inplace=True)

In [12]:
df_encoded.isna().sum()

onehot__contrato_Staff (planta permanente)    0
onehot__dedicacion_Full-Time                  0
remainder__salario                            0
remainder__anos_de_experiencia                0
remainder__antiguedad_en_la_empresa_actual    0
remainder__anos_en_el_puesto_actual           0
remainder__cuantas_personas_tenes_a_cargo     0
remainder__edad                               0
remainder__marvin_rol                         0
remainder__cantidad_personas_encoded          0
remainder__modalidad_encoded                  0
remainder__seniority_encoded                  0
dtype: int64

In [13]:
df_encoded.rename(columns={"onehot__contrato_Staff (planta permanente)": "contrato", "onehot__dedicacion_Full-Time": "dedicacion"}, inplace=True)	

In [14]:
df_encoded.to_csv("data/tech_salaries_encoded.csv", index=False)